In [ ]:
%load_ext autoreload
%autoreload 2

import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import formulas
import pandas as pd
import random
import dataframes

plotly.offline.init_notebook_mode(connected=True)

# Inputs
Assume fixed rate mortgage

In [ ]:
# Loan
DOWN_PAYMENT = 200000
PRINCIPAL = 900000 # This is the value of house. Principal is probably a bad name for this.
LOAN_APR = 0.077
DURATION = 30 # number of years

# Additional Costs
# Home insurance 1500 per year
# HOA 150 per month
ADDITIONAL_MONTHLY_COSTS = 1500/12 + 150

# Tax
TAX_APPRAISAL_VAL = PRINCIPAL
TAX_APR = 0.02
TAX_APPRAISAL_VAL_GROWTH_RATE = 0.0

# Income
MONTHLY_INCOME = 200000/12 # Gross monthly income because debt-to-income ratio is calculated from gross

# Don't touch
PAYMENTS_N = 12 # monthly payments. CAUTION - this should probably never be changed because I'm sure I've hardcoded 12 elsewhere
loan_value  = PRINCIPAL - DOWN_PAYMENT

# Property Tax

<span style="color:red">
</span>

In [ ]:
tax_df = dataframes.property_tax_amortization(TAX_APPRAISAL_VAL, TAX_APR, DURATION, TAX_APPRAISAL_VAL_GROWTH_RATE)
# display(tax_df.round(0))

# Mortgage Amortization

In [ ]:
ma_df = dataframes.mortgage_amortization(loan_value, LOAN_APR, PAYMENTS_N, DURATION, DOWN_PAYMENT)
display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()

# Payment Amortization

In [ ]:
as_df = dataframes.amortization_summary(ma_df, tax_df, monthly_income=MONTHLY_INCOME, monthly_additional_costs=ADDITIONAL_MONTHLY_COSTS)
display(as_df.round(0))

#Sums
fig = px.bar(as_df, x=as_df.index, y=['principal paid', 'interest paid', 'tax paid'], title='Amounts Paid')
fig.add_scatter(x=as_df.index, y=as_df['mortgage balance'], name='mortgage balance', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['equity'], name='equity', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['payments'], name='payments', mode='lines')
fig.add_scatter(x=as_df.index, y=as_df['appraisal value'], name='appraisal value', mode='lines')
fig.show()


# Monthly Payments

In [ ]:
fig = px.bar(as_df, x=as_df.index, y=['mortgage payment principal', 'mortgage payment interest', 'monthly tax payment'], title='Monthly Payments')
fig.add_scatter(x=as_df.index, y=as_df['monthly payment'], name='monthly payment', mode='lines')
fig.show()

# Debt to Income Ratio

In [ ]:
fig = px.scatter(as_df, x=as_df.index, y=['debt to income ratio'], title='Debt to Income Ratio')
fig.update_yaxes(range=[0, 100])
fig.show()

# Randomized Interest Rate
Randomize the interest rate every few years. Re-finance the loan into this new rate regardless of whether it's higher or lower.

In [ ]:
INT_RANGE_LOW = 0.02
INT_RANGE_HI = 0.11
INT_CHANGE_PERIOD_YEARS = 3 # randomize each number of years

def randomized_interest(starting_interest_rate, min, max, change_period_years, duration_months):
    periodic_interest_rates = [starting_interest_rate]
    change_period_months = 12 * change_period_years
    t = np.arange(duration_months)
    interest = np.zeros(t.size)
    month_slice_start = 0
    month_slice_end = change_period_months
    interest[month_slice_start:month_slice_start + change_period_months] = starting_interest_rate
    for i in range(duration_months // 12 // change_period_years - 1):
        month_slice_start = (i + 1) * change_period_months
        new_int = random.uniform(INT_RANGE_LOW, INT_RANGE_HI)
        periodic_interest_rates.append(new_int)
        interest[month_slice_start:month_slice_start + change_period_months] = new_int
    df = pd.DataFrame(
    {
        'month': t,
        'interest rate': interest
    })
    df = df.set_index('month')
    return df, periodic_interest_rates
    
ri_df, periodic_interest_rates = randomized_interest(LOAN_APR, INT_RANGE_LOW, INT_RANGE_HI, INT_CHANGE_PERIOD_YEARS, DURATION * PAYMENTS_N)
fig = px.scatter(ri_df, x=ri_df.index, y=['interest rate'], title='Randomized Interest Rate')
fig.show()
print(periodic_interest_rates)

In [ ]:
ma_df = dataframes.refi_mortgage_amortization(PRINCIPAL, PAYMENTS_N, DURATION, DOWN_PAYMENT, INT_CHANGE_PERIOD_YEARS, periodic_interest_rates)
display(ma_df.round(0))

fig = px.bar(ma_df, x=ma_df.index, y=['interest paid', 'principal paid'])
fig.add_scatter(x=ma_df.index, y=ma_df['mortgage balance'], name='mortgage balance')
fig.add_scatter(x=ma_df.index, y=ma_df['equity'], name='equity')
fig.show()